In [1]:
import sage
from sklearn.model_selection import train_test_split


# Load data
df = sage.datasets.credit()

# Feature names and categorical columns (for CatBoost model)
feature_names = df.columns.tolist()[:-1]
categorical_columns = [
    'Checking Status', 'Credit History', 'Purpose', 'Credit Amount',
    'Savings Account/Bonds', 'Employment Since', 'Personal Status',
    'Debtors/Guarantors', 'Property Type', 'Other Installment Plans',
    'Housing Ownership', 'Job', 'Telephone', 'Foreign Worker'
]
categorical_inds = [feature_names.index(col) for col in categorical_columns]

# Split data
train, test = train_test_split(
    df.values, test_size=int(0.1 * len(df.values)), random_state=0)
train, val = train_test_split(
    train, test_size=int(0.1 * len(df.values)), random_state=0)
Y_train = train[:, -1].copy().astype(int)
Y_val = val[:, -1].copy().astype(int)
Y_test = test[:, -1].copy().astype(int)
train = train[:, :-1].copy()
val = val[:, :-1].copy()
test = test[:, :-1].copy()

In [2]:
import os

from catboost import CatBoostClassifier


model_filename = "../credit_model.cbm"

if os.path.isfile(model_filename):
    model = CatBoostClassifier()
    model.load_model(model_filename)
else:
    model = CatBoostClassifier(iterations=50, learning_rate=0.3, depth=3)
    model = model.fit(train, Y_train, categorical_inds, eval_set=(val, Y_val), verbose=False)
    model.save_model(model_filename)

In [3]:
import numpy as np
from sklearn.metrics import log_loss


# Calculate performance
p = np.array([np.sum(Y_train == i) for i in np.unique(Y_train)]) / len(Y_train)
base_ce = log_loss(Y_test.astype(int), p[np.newaxis].repeat(len(test), 0))
ce = log_loss(Y_test.astype(int), model.predict_proba(test))

print('Base rate cross entropy = {:.3f}'.format(base_ce))
print('Model cross entropy = {:.3f}'.format(ce))

Base rate cross entropy = 0.602
Model cross entropy = 0.457


In [4]:
from sklearn.metrics import roc_auc_score

print("ROC AUC Score = {:.3f}".format(roc_auc_score(Y_test, model.predict_proba(test)[:, 1])))

ROC AUC Score = 0.849


In [5]:
from sklearn.metrics import confusion_matrix

pred_test = model.predict_proba(test)
pred_test = np.argmax(pred_test, axis=1)

tn, fp, fn, tp = confusion_matrix(Y_test, pred_test, labels=[0,1]).ravel()
p = tp + fn
fnr = fn / p if p > 0.0 else np.float64(0.0)

print("False negative rate = {:.3f}".format(fnr))

False negative rate = 0.042


In [6]:
print("Classifier got {} positive examples wrong".format(int(round(Y_test.sum() * fnr))))

Classifier got 3 positive examples wrong


# Global FNR

In [7]:
# Setup and calculate with custom fairness-related loss function
imputer = sage.MarginalImputer(model, train[:512])
estimator_fnr = sage.PermutationEstimator(imputer, 'fnr')
%time sage_values_fnr = estimator_fnr(test, Y_test, verbose=True)

# Print results
print("SAGE values using false negative rate as loss:", sage_values_fnr)

  0%|          | 0/1 [00:00<?, ?it/s]

fn = 0, p = 373, fnr = 0.0
===> Iteration 0, Prev loss = 0.0 [outer loop]
fn = 0, p = 373, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 0, p = 373, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 0, p = 373, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 2, p = 373, fnr = 0.005361930294906166
	Loss = 0.005361930294906166 [inner loop]
fn = 5, p = 373, fnr = 0.013404825737265416
	Loss = 0.013404825737265416 [inner loop]
fn = 5, p = 373, fnr = 0.013404825737265416
	Loss = 0.013404825737265416 [inner loop]
fn = 7, p = 373, fnr = 0.01876675603217158
	Loss = 0.01876675603217158 [inner loop]
fn = 7, p = 373, fnr = 0.01876675603217158
	Loss = 0.01876675603217158 [inner loop]
fn = 6, p = 373, fnr = 0.0160857908847185
	Loss = 0.0160857908847185 [inner loop]
fn = 6, p = 373, fnr = 0.0160857908847185
	Loss = 0.0160857908847185 [inner loop]
fn = 8, p = 373, fnr = 0.021447721179624665
	Loss = 0.021447721179624665 [inner loop]
fn = 11, p = 373, fnr = 0.029490616621983913
	Loss = 0.029490616621983913 [inner loop]
fn = 9,

fn = 11, p = 357, fnr = 0.03081232492997199
	Loss = 0.03081232492997199 [inner loop]
fn = 9, p = 357, fnr = 0.025210084033613446
	Loss = 0.025210084033613446 [inner loop]
fn = 10, p = 357, fnr = 0.028011204481792718
	Loss = 0.028011204481792718 [inner loop]
fn = 13, p = 357, fnr = 0.036414565826330535
	Loss = 0.036414565826330535 [inner loop]
fn = 12, p = 357, fnr = 0.03361344537815126
	Loss = 0.03361344537815126 [inner loop]
StdDev Ratio = 0.2433 (Converge at 0.0250)
fn = 0, p = 362, fnr = 0.0
===> Iteration 5, Prev loss = 0.0 [outer loop]
fn = 0, p = 362, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 0, p = 362, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 1, p = 362, fnr = 0.0027624309392265192
	Loss = 0.0027624309392265192 [inner loop]
fn = 1, p = 362, fnr = 0.0027624309392265192
	Loss = 0.0027624309392265192 [inner loop]
fn = 1, p = 362, fnr = 0.0027624309392265192
	Loss = 0.0027624309392265192 [inner loop]
fn = 1, p = 362, fnr = 0.0027624309392265192
	Loss = 0.0027624309392265192 [inner loo

fn = 6, p = 376, fnr = 0.015957446808510637
	Loss = 0.015957446808510637 [inner loop]
fn = 8, p = 376, fnr = 0.02127659574468085
	Loss = 0.02127659574468085 [inner loop]
fn = 10, p = 376, fnr = 0.026595744680851064
	Loss = 0.026595744680851064 [inner loop]
fn = 11, p = 376, fnr = 0.02925531914893617
	Loss = 0.02925531914893617 [inner loop]
fn = 14, p = 376, fnr = 0.03723404255319149
	Loss = 0.03723404255319149 [inner loop]
fn = 18, p = 376, fnr = 0.047872340425531915
	Loss = 0.047872340425531915 [inner loop]
fn = 19, p = 376, fnr = 0.05053191489361702
	Loss = 0.05053191489361702 [inner loop]
fn = 21, p = 376, fnr = 0.05585106382978723
	Loss = 0.05585106382978723 [inner loop]
fn = 22, p = 376, fnr = 0.05851063829787234
	Loss = 0.05851063829787234 [inner loop]
fn = 25, p = 376, fnr = 0.06648936170212766
	Loss = 0.06648936170212766 [inner loop]
fn = 24, p = 376, fnr = 0.06382978723404255
	Loss = 0.06382978723404255 [inner loop]
StdDev Ratio = 0.2166 (Converge at 0.0250)
fn = 0, p = 353, f

fn = 1, p = 364, fnr = 0.0027472527472527475
	Loss = 0.0027472527472527475 [inner loop]
fn = 2, p = 364, fnr = 0.005494505494505495
	Loss = 0.005494505494505495 [inner loop]
fn = 2, p = 364, fnr = 0.005494505494505495
	Loss = 0.005494505494505495 [inner loop]
fn = 2, p = 364, fnr = 0.005494505494505495
	Loss = 0.005494505494505495 [inner loop]
fn = 3, p = 364, fnr = 0.008241758241758242
	Loss = 0.008241758241758242 [inner loop]
fn = 7, p = 364, fnr = 0.019230769230769232
	Loss = 0.019230769230769232 [inner loop]
fn = 7, p = 364, fnr = 0.019230769230769232
	Loss = 0.019230769230769232 [inner loop]
fn = 5, p = 364, fnr = 0.013736263736263736
	Loss = 0.013736263736263736 [inner loop]
fn = 7, p = 364, fnr = 0.019230769230769232
	Loss = 0.019230769230769232 [inner loop]
fn = 8, p = 364, fnr = 0.02197802197802198
	Loss = 0.02197802197802198 [inner loop]
fn = 6, p = 364, fnr = 0.016483516483516484
	Loss = 0.016483516483516484 [inner loop]
fn = 8, p = 364, fnr = 0.02197802197802198
	Loss = 0.0

...

fn = 5, p = 327, fnr = 0.01529051987767584
	Loss = 0.01529051987767584 [inner loop]
fn = 6, p = 327, fnr = 0.01834862385321101
	Loss = 0.01834862385321101 [inner loop]
fn = 7, p = 327, fnr = 0.021406727828746176
	Loss = 0.021406727828746176 [inner loop]
fn = 8, p = 327, fnr = 0.024464831804281346
	Loss = 0.024464831804281346 [inner loop]
fn = 12, p = 327, fnr = 0.03669724770642202
	Loss = 0.03669724770642202 [inner loop]
fn = 13, p = 327, fnr = 0.039755351681957186
	Loss = 0.039755351681957186 [inner loop]
fn = 13, p = 327, fnr = 0.039755351681957186
	Loss = 0.039755351681957186 [inner loop]
fn = 12, p = 327, fnr = 0.03669724770642202
	Loss = 0.03669724770642202 [inner loop]
fn = 12, p = 327, fnr = 0.03669724770642202
	Loss = 0.03669724770642202 [inner loop]
fn = 10, p = 327, fnr = 0.03058103975535168
	Loss = 0.03058103975535168 [inner loop]
fn = 11, p = 327, fnr = 0.03363914373088685
	Loss = 0.03363914373088685 [inner loop]
fn = 14, p = 327, fnr = 0.04281345565749235
	Loss = 0.0428134

fn = 0, p = 354, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 0, p = 354, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 0, p = 354, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 0, p = 354, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 0, p = 354, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 2, p = 354, fnr = 0.005649717514124294
	Loss = 0.005649717514124294 [inner loop]
fn = 3, p = 354, fnr = 0.00847457627118644
	Loss = 0.00847457627118644 [inner loop]
fn = 3, p = 354, fnr = 0.00847457627118644
	Loss = 0.00847457627118644 [inner loop]
fn = 2, p = 354, fnr = 0.005649717514124294
	Loss = 0.005649717514124294 [inner loop]
fn = 3, p = 354, fnr = 0.00847457627118644
	Loss = 0.00847457627118644 [inner loop]
fn = 7, p = 354, fnr = 0.01977401129943503
	Loss = 0.01977401129943503 [inner loop]
fn = 9, p = 354, fnr = 0.025423728813559324
	Loss = 0.025423728813559324 [inner loop]
fn = 11, p = 354, fnr = 0.031073446327683617
	Loss = 0.031073446327683617 [inner loop]
fn = 15, p = 354, fnr = 0.0423728813559322
	Loss = 0.042372

fn = 13, p = 368, fnr = 0.035326086956521736
	Loss = 0.035326086956521736 [inner loop]
StdDev Ratio = 0.0886 (Converge at 0.0250)
fn = 0, p = 343, fnr = 0.0
===> Iteration 1259, Prev loss = 0.0 [outer loop]
fn = 0, p = 343, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 0, p = 343, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 0, p = 343, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 0, p = 343, fnr = 0.0
	Loss = 0.0 [inner loop]
fn = 1, p = 343, fnr = 0.0029154518950437317
	Loss = 0.0029154518950437317 [inner loop]
fn = 1, p = 343, fnr = 0.0029154518950437317
	Loss = 0.0029154518950437317 [inner loop]
fn = 1, p = 343, fnr = 0.0029154518950437317
	Loss = 0.0029154518950437317 [inner loop]
fn = 1, p = 343, fnr = 0.0029154518950437317
	Loss = 0.0029154518950437317 [inner loop]
fn = 2, p = 343, fnr = 0.0058309037900874635
	Loss = 0.0058309037900874635 [inner loop]
fn = 5, p = 343, fnr = 0.014577259475218658
	Loss = 0.014577259475218658 [inner loop]
fn = 5, p = 343, fnr = 0.014577259475218658
	Loss = 0.01457

KeyboardInterrupt: 

NameError: name 'sage_values_fnr' is not defined

In [ ]:
sage_values_fnr.plot(feature_names, title='Feature Importance with respect to Global False Negative Rate')

# Young group FNR

In [ ]:
age_col = feature_names.index("Age")

In [ ]:
where_young_train = train[:,age_col] <= 30
where_young_test = test[:,age_col] <= 30

test_young = test[where_young_test]
Y_test_young = Y_test[where_young_test]

In [ ]:
# Setup and calculate with custom fairness-related loss function
imputer_young = sage.MarginalImputer(model, train[young_train])
estimator_young_fnr = sage.PermutationEstimator(imputer_young, 'fnr')
%time sage_values_young_fnr = estimator_young_fnr(test_young, Y_test_young, verbose=True)

# Print results
print("SAGE values using false negative rate as loss:", sage_values_young_fnr)

In [ ]:
sage_values_young_fnr.plot(feature_names, title='Feature Importance with respect to Young Group\'s False Negative Rate')

# Old group FNR

In [ ]:
where_old_train = train[:,age_col] > 30
where_old_test = np.invert(where_young_test)

test_old = test[where_old_test]
Y_test_old = Y_test[where_old_test]

In [ ]:
# Setup and calculate with custom fairness-related loss function
imputer_old = sage.MarginalImputer(model, train[where_old_train])
estimator_old_fnr = sage.PermutationEstimator(imputer_old, 'fnr')
%time sage_values_old_fnr = estimator_old_fnr(test_old, Y_test_old, verbose=True)

# Print results
print("SAGE values using false negative rate as loss:", sage_values_old_fnr)

In [ ]:
sage_values_old_fnr.plot(feature_names, title='Feature Importance with respect to Old Group\'s False Negative Rate')